In [19]:
import joblib
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import LeaveOneOut
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
# from envs.filtr_json_from_race import load_from_db
import sqlite3
import json
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
class RaceModel(nn.Module):
    def __init__(self,input_size,output_size):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Linear(input_size,128), nn.ReLU(),
            nn.Linear(128,128), nn.ReLU()
        )
        self.out = nn.Linear(128,output_size)     
        
        self.scaler_X = None
        self.scaler_Y = None

    def forward(self, x):
        h = self.shared(x)
        return self.out(h)
    
    def set_scalers(self, scaler_X, scaler_Y):
        self.scaler_X = scaler_X
        self.scaler_Y = scaler_Y
    
    def scale_input(self, X_grouped,Y_grouped):
         # --- Continous and discrete feature indices ---
        cont_indices_x = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]     # continuous
        cont_indices_y = [0,1,2,3,4,5,6,7,8,9,10,11]          # continuous
    

        # --- Scale continuous features ---
        all_X_cont = np.vstack([x[:, cont_indices_x] for x in X_grouped])
        all_Y_cont = np.vstack([y[:, cont_indices_y] for y in Y_grouped])
        self.scaler_X = StandardScaler().fit(all_X_cont)
        self.scaler_Y = StandardScaler().fit(all_Y_cont)

        # Scale X and Y for continuous features
        X_scaled_grouped = []
        Y_scaled_grouped = []
        for x_seq, y_seq in zip(X_grouped, Y_grouped):
            x_scaled = np.array(x_seq)
            x_scaled[:, cont_indices_x] = self.scaler_X.transform(x_seq[:, cont_indices_x])
            X_scaled_grouped.append(x_scaled)

            y_scaled = np.array(y_seq)
            y_scaled[:, cont_indices_y] = self.scaler_Y.transform(y_seq[:, cont_indices_y])
            Y_scaled_grouped.append(y_scaled)

        # Conversion to torch tensors
        X_t = [torch.tensor(x, dtype=torch.float32) for x in X_scaled_grouped]
        Y_cont_t = [torch.tensor(y[:, cont_indices_y], dtype=torch.float32) for y in Y_scaled_grouped]

        return X_t, Y_cont_t

In [ ]:
def load_data_from_db():
    
    """
    Load data so that each race is a separate sequence:
    X = [ [state1_race1, state2_race1, ...], [state1_race2, ...] ]
    Y = [ [next1_race1, next2_race1, ...], ... ]
    """
    conn = sqlite3.connect(
        "E:/pracadyp/Race-optimization-reinforcement-learning/data/db_states_for_regress/race_data_states.db"
    )
    cursor = conn.cursor()
    cursor.execute("SELECT race_id, states_json FROM races ORDER BY race_id")
    rows = cursor.fetchall()
    conn.close()

    X_grouped, Y_grouped = [], []

    data = []

    for race_id, states_json in rows:
        states = json.loads(states_json)
        data.append(states)
    
    return data



In [4]:
def make_single_step(X_grouped, Y_grouped):
    X_single, Y_single = [], []
    for X_race, Y_race in zip(X_grouped, Y_grouped):
        for t in range(len(X_race)):
            X_single.append(X_race[t])
            Y_single.append(Y_race[t])
    return X_single, Y_single

In [ ]:
X, Y = load_data_from_db()

input_size = X[0].shape[1]
output_size = Y[0].shape[1]
loo = LeaveOneOut()
all_fold_train = []
all_fold_test = []
all_fold_r2_test = []
for fold, (train_idx, test_idx) in enumerate(loo.split(X)):
    

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)
    model = RaceModel(input_size, output_size).to(device)

    X_scaled, Y_scaled = model.scale_input(X, Y)

    X_train_single, Y_train_single = make_single_step([X_scaled[i] for i in train_idx],
                                                      [Y_scaled[i] for i in train_idx])
    X_test_single, Y_test_single = make_single_step([X_scaled[i] for i in test_idx],
                                                    [Y_scaled[i] for i in test_idx])
    
    X_train_tensor = torch.tensor(X_train_single, dtype=torch.float32).to(device)
    Y_train_tensor = torch.tensor(Y_train_single, dtype=torch.float32).to(device)
    X_test_tensor = torch.tensor(X_test_single, dtype=torch.float32).to(device)
    Y_test_tensor = torch.tensor(Y_test_single, dtype=torch.float32).to(device)

    train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)

    optimizer = optim.Adam(model.parameters(), lr=4e-5)
    loss_cont = nn.MSELoss()
    loss_cat  = nn.CrossEntropyLoss()

    fold_train_losses = []
    fold_test_losses = []

    print(f"Fold {fold+1}")
    X_train = [X_t[i] for i in train_idx]
    X_test  = [X_t[i] for i in test_idx]
    Y_cont_train = [Y_t[i] for i in train_idx]
    Y_cont_test  = [Y_t[i] for i in test_idx]
 

    X_train = [x.to(device) for x in X_train]
    Y_train = [y.to(device) for y in Y_train]
    

    X_test = [x.to(device) for x in X_test]
    Y_test = [y.to(device) for y in Y_test]   
    
    
    n_epochs = 750
    for epoch in range(n_epochs):
        total_loss = 0.0
        model.train()
        for x_seq, y_seq in zip(X_train, Y_train): # y_cat_seq, Y_cat_train):
           
            optimizer.zero_grad()

            y_pred = model(x_seq) # continuous and list of categorical predictions , cat_preds

            loss = loss_cont(y_pred, y_seq)
            
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_train_loss = total_loss / len(X_train)

        total_test_loss = 0.0
        model.eval()
        all_mse = []

        y_true_all = []
        y_pred_all = []


        with torch.no_grad():
            for x_seq, y_seq in zip(X_test, Y_test): #,y_cat_seq, Y_cat_test):
                x_seq = x_seq.to(device)
                y_seq = y_seq.to(device)
                # y_cat_seq = [y.to(device) for y in y_cat_seq]

                cont_pred = model(x_seq) #, cat_preds = model(x_seq)

                loss = loss_cont(cont_pred, y_seq)

                total_test_loss += loss.item()
                

        if epoch == n_epochs-1:
            all_r2_per_output = []
            with torch.no_grad():
                for x_seq, y_seq in zip(X_test, Y_test):
                    y_pred = model(x_seq.to(device))
                    y_seq_orig = model.scaler_Y.inverse_transform(y_seq.cpu().numpy())
                    y_pred_orig = model.scaler_Y.inverse_transform(y_pred.cpu().numpy())
                    y_true_all.append(y_seq_orig)
                    y_pred_all.append(y_pred_orig)

                    r2s = [r2_score(y_seq_orig[:, i], y_pred_orig[:, i]) for i in range(y_seq_orig.shape[1])]
                    all_r2_per_output.append(r2s)

            avg_r2_per_output = np.mean(all_r2_per_output, axis=0)
            print("Avg R2 per output:", "{:.4f}".format(avg_r2_per_output))
            print("Mean R2:", "{:.4f}".format(np.mean(avg_r2_per_output)))

        avg_test_loss = total_test_loss / len(X_test)
     
        
        fold_train_losses.append(avg_train_loss)
        fold_test_losses.append(avg_test_loss)

    
        
    all_fold_train.append(fold_train_losses)
    all_fold_test.append(fold_test_losses)

   

for fold in range(len(all_fold_train)):
    plt.figure()
    plt.plot(all_fold_train[fold], label='Train')
    plt.plot(all_fold_test[fold], label='Test')
    plt.title(f'Fold {fold+1}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

Using device: cuda


NameError: name 'X_train_single' is not defined